In [6]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels as sm
import geopandas as gpd


In [ ]:
#load data
child_data_raw = pd.read_csv("child_w5_anon_v1.0.0-stata11.csv")